# Libraries

In [23]:
import requests
import bs4 as BeautifulSoup
import pandas as pd
import sqlite3
import numpy as np
from datetime import datetime

# Extraction

In [34]:
url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
Table_Att = ['Name', 'MC_USD_Billion']

def extract(url, Table_Att):
    page = pd.read_html(url)
    print(page)
    data = page[1]
    del data['Rank']
    data.columns = Table_Att
    return pd.DataFrame(data)
    print(data)    

In [36]:
extract(url, Table_Att)

[      0     1     2
0   Aug   SEP   Oct
1   NaN    08   NaN
2  2022  2023  2024,    Rank                                Bank name  Market cap (US$ billion)
0     1                           JPMorgan Chase                    432.92
1     2                          Bank of America                    231.52
2     3  Industrial and Commercial Bank of China                    194.56
3     4               Agricultural Bank of China                    160.68
4     5                                HDFC Bank                    157.91
5     6                              Wells Fargo                    155.87
6     7                        HSBC Holdings PLC                    148.90
7     8                           Morgan Stanley                    140.83
8     9                  China Construction Bank                    139.82
9    10                            Bank of China                    136.81,     Rank                                        Bank name  \
0      1  Industrial and Commer

,Name,MC_USD_Billion
0,JPMorgan Chase,432.92
1,Bank of America,231.52
2,Industrial and Commercial Bank of China,194.56
3,Agricultural Bank of China,160.68
4,HDFC Bank,157.91
5,Wells Fargo,155.87
6,HSBC Holdings PLC,148.90
7,Morgan Stanley,140.83
8,China Construction Bank,139.82
9,Bank of China,136.81


# Transform

In [26]:
exchange_rate = pd.read_csv("Desktop/exchange_rate.csv")
dict = exchange_rate.set_index('Currency').to_dict()['Rate']
def transform(data):
    data['MC_EUR_Billion'] = [np.round(x*dict['EUR'],2) for x in data['MC_USD_Billion']]
    data['MC_GBP_Billion'] = [np.round(x*dict['GBP'],2) for x in data['MC_USD_Billion']]
    data['MC_INR_Billion'] = [np.round(x*dict['INR'],2) for x in data['MC_USD_Billion']]
    return data

In [27]:
transform(data)

,Name,MC_USD_Billion,MC_EUR_Billion,MC_GBP_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,402.62,346.34,35910.71
1,Bank of America,231.52,215.31,185.22,19204.58
2,Industrial and Commercial Bank of China,194.56,180.94,155.65,16138.75
3,Agricultural Bank of China,160.68,149.43,128.54,13328.41
4,HDFC Bank,157.91,146.86,126.33,13098.63
5,Wells Fargo,155.87,144.96,124.70,12929.42
6,HSBC Holdings PLC,148.90,138.48,119.12,12351.26
7,Morgan Stanley,140.83,130.97,112.66,11681.85
8,China Construction Bank,139.82,130.03,111.86,11598.07
9,Bank of China,136.81,127.23,109.45,11348.39


In [28]:
# answer to quiz question data['MC_EUR_Billion'][4]

# Loading

In [29]:
csv_path = "/Users/jameelbrannon/Largest_banks_data.csv"
def load_to_csv(data, csv_path):
    data.to_csv(csv_path)

table_name = "Largest_banks"
sql_connection = sqlite3.connect('Banks.db')

def load_to_db(data, sql_connection, table_name):
    data.to_sql(table_name, sql_connection, if_exists = "replace", index = False)
load_to_db(data, sql_connection, table_name)

#query_statement = f"SELECT * from {table_name}"
query_statement = f"SELECT AVG(MC_GBP_BILLION) from {table_name}"
#query_statement = f"SELECT Name from {table_name} limit 5"

def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

def log_progress(message):
    ''' This function logs the mentioned message at a given stage of the 
    code execution to a log file. Function returns nothing.'''
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open("./code_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')    

log_progress('Preliminaries complete. Initiating ETL process')
data = extract(url, Table_Att)
log_progress('Data extraction complete. Initiating Transformation process')
data = transform(data)
log_progress('Data transformation complete. Initiating loading process')
load_to_csv(data, csv_path)
log_progress('Data saved to CSV file')
sql_connection = sqlite3.connect('Banks.db')
log_progress('SQL Connection initiated.')
load_to_db(data, sql_connection, table_name)
log_progress('Data loaded to Database as table. Running the query')
run_query(query_statement, sql_connection)
log_progress('Process Complete.')
sql_connection.close()

SELECT AVG(MC_GBP_BILLION) from Largest_banks
   AVG(MC_GBP_BILLION)
0              151.987


# Queries

In [ ]:
run_query(query_statement, sql_connection)

In [31]:
3/5*100

60.0

In [32]:
data['MC_EUR_Billion'][4]

146.86

In [ ]:
run_query(query_statement, sql_connection)

# Log Progress

In [33]:
data

,Name,MC_USD_Billion,MC_EUR_Billion,MC_GBP_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,402.62,346.34,35910.71
1,Bank of America,231.52,215.31,185.22,19204.58
2,Industrial and Commercial Bank of China,194.56,180.94,155.65,16138.75
3,Agricultural Bank of China,160.68,149.43,128.54,13328.41
4,HDFC Bank,157.91,146.86,126.33,13098.63
5,Wells Fargo,155.87,144.96,124.70,12929.42
6,HSBC Holdings PLC,148.90,138.48,119.12,12351.26
7,Morgan Stanley,140.83,130.97,112.66,11681.85
8,China Construction Bank,139.82,130.03,111.86,11598.07
9,Bank of China,136.81,127.23,109.45,11348.39
